In [21]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.io as pio
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Constants
API_KEY = ''  # Replace with your actual API key
#API_KEY = os.getenv('GLASSNODE_API_KEY')
SINCE_DATE = int(datetime(2020, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/supply/profit_relative',
    'https://api.glassnode.com/v1/metrics/indicators/net_unrealized_profit_loss',
    'https://api.glassnode.com/v1/metrics/indicators/realized_profit_loss_ratio'
    
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [22]:
def calculate_smoothed_bands(df, column = 'profit_relative', smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = df[column]
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [23]:
def calculate_smoothed_bands_v5(df, column = 'realized_profit_loss_ratio', smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = np.log(df[column])
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [24]:
merged_df['supplyinprofit_sma_percentile'], merged_df['supplyinprofit_upper'], merged_df['supplyinprofit_lower']= calculate_smoothed_bands(merged_df, column='profit_relative',smoothing_fun='sma', smoothing_window=7, range_method='percentile', range_window=180)
merged_df['nupl_sma_zscore'], merged_df['nupl_upper'], merged_df['nupl_lower']= calculate_smoothed_bands(merged_df, column='net_unrealized_profit_loss',smoothing_fun='sma', smoothing_window=7, range_method='zscore', range_window=180)
merged_df['plratio_sma_zscore'], merged_df['plratio_upper'], merged_df['plratio_lower']= calculate_smoothed_bands_v5(merged_df, column = 'realized_profit_loss_ratio', smoothing_fun='sma', smoothing_window=7, range_method='zscore', range_window=180)

In [25]:
import pandas as pd
from plotly.subplots import make_subplots

def create_indicator_chart(merged_df, indicator_column, chart_title, bands="on", upper_lower=["fee_upper", "fee_lower"]):
   # Filter data for the last year
   one_year_ago = datetime.now() - timedelta(days=730)
   merged_df_last_year = merged_df[merged_df.index > one_year_ago]

   # Define colors
   GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
   BLUE_COLOR = 'rgb(0, 0, 255)'  # Blue
   GREEN_COLOR = 'rgb(0, 255, 0)'  # Green for upper band
   RED_COLOR = 'rgb(255, 0, 0)'    # Red for lower band

   # Create the visualization
   fig = make_subplots(specs=[[{"secondary_y": True}]])

   # Add price trace
   fig.add_trace(
       go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], 
                 name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
       secondary_y=False,
   )

   # Add indicator trace
   indicator = merged_df_last_year[indicator_column]
   fig.add_trace(
       go.Scatter(
           x=merged_df_last_year.index,
           y=indicator,
           name=indicator_column,
           line=dict(color=BLUE_COLOR, width=2),
           mode='lines'
       ),
       secondary_y=True,
   )

   # Add band traces if enabled
   if bands == "on":
       # Simply use the provided column names directly
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[0]], 
               name="Upper Band",
               line=dict(color=GREEN_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[1]], 
               name="Lower Band",
               line=dict(color=RED_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )

   # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
   for month in [1, 4, 7, 10]:
       for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
           date = pd.Timestamp(year=year, month=month, day=1)
           if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
               fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

   # Get the last value of the indicator and format it
   last_value = indicator.iloc[-1]
   
   # Function to format number to K, M, B
   def format_number(num):
       if abs(num) >= 1e9:
           return f"{num/1e9:.2f}B"
       elif abs(num) >= 1e6:
           return f"{num/1e6:.2f}M"
       elif abs(num) >= 1e3:
           return f"{num/1e3:.2f}K"
       else:
           return f"{num:.2f}"
   
   formatted_value = format_number(last_value)

   # Add annotation for the last value
   fig.add_annotation(
       x=0.95,  
       y=last_value*0.88,  
       xref="paper",
       yref="y2", 
       text=formatted_value,
       showarrow=False,
       font=dict(size=18, color=BLUE_COLOR),
       align="left",
       xanchor="left",
       yanchor="middle",
   )

   # Update layout
   fig.update_layout(
       title={
           'text': chart_title,
           'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
       },
       showlegend=False,  
       hovermode="x unified",
       plot_bgcolor='white',
       paper_bgcolor='white',
       font={'color': 'black', 'size': 14},
       width=1000,  
       height=450,  
   )

   # Update axes
   fig.update_xaxes(
       showgrid=False, 
       tickfont={'color': 'black', 'size': 14},
       zeroline=False,
       title_text=''  
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=False, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18)
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=True, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       side='right',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18),
       autorange=True
   )

   return fig

In [26]:
# Create charts for each indicator
indicators = [
    ('supplyinprofit_sma_percentile',"Bitcoin: Percent Supply in Profit", "on", ["supplyinprofit_upper", "supplyinprofit_lower"]),
    ('nupl_sma_zscore',"Bitcoin: NUPL", "on", ["nupl_upper", "nupl_lower"]),
    ('plratio_sma_zscore',"Bitcoin: Realized Profit/Loss Ratio", "on", ["plratio_upper", "plratio_lower"])
]

for indicator, title,  bands, upper_lower in indicators:
    fig = create_indicator_chart(merged_df, indicator, title, bands, upper_lower)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [27]:
merged_df.to_csv('bitcoin_analysis_data_liquidity2.csv', index=True)

In [28]:
merged_df.tail()

,price_usd_close,profit_relative,net_unrealized_profit_loss,realized_profit_loss_ratio,supplyinprofit_sma_percentile,supplyinprofit_upper,supplyinprofit_lower,nupl_sma_zscore,nupl_upper,nupl_lower,plratio_sma_zscore,plratio_upper,plratio_lower
t,,,,,,,,,,,,,
2024-11-27,95981.580554,0.960769,0.617351,17.182364,0.965515,0.985589,0.753546,0.621299,0.556397,0.467159,3.036682,2.535262,0.395302
2024-11-28,95651.196450,0.960222,0.614898,15.369293,0.960163,0.985589,0.753546,0.618409,0.557259,0.466923,2.687637,2.540562,0.395546
2024-11-29,97447.132368,0.966966,0.620244,21.349427,0.956820,0.985589,0.753546,0.616492,0.558099,0.466709,2.520279,2.543958,0.395864
2024-11-30,96514.052710,0.955690,0.616435,11.692875,0.955512,0.985589,0.753546,0.614916,0.558923,0.466514,2.359380,2.545945,0.396167
2024-12-01,97311.455238,0.966091,0.619183,19.780821,0.955457,0.985589,0.753546,0.613837,0.559703,0.466333,2.445292,2.550371,0.396982


In [29]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Change the grey color to a darker shade
GREY_COLOR = 'rgba(70, 70, 70, 0.9)'  # Darker, more opaque grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line (darker)
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color=GREY_COLOR, width=3),  # Using darker grey
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),  # Using darker grey
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")  # Using darker grey
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},  # Using darker grey
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}  # Using darker grey
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
            side='left',
            title_standoff=35
        ),
        height=400 + (len(metrics_titles) * 30),
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),
        font=dict(color=GREY_COLOR, size=14)  # Using darker grey
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [30]:
# Manually define metrics and their titles
metrics_titles = {

    'supplyinprofit_sma_percentile': "Percent Supply in Profit",
    'nupl_sma_zscore':"NUPL",
    'plratio_sma_zscore': "Realized Profit/Loss Ratio"
    
    # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_profitloss2_raw_metrics_dot_plot.html')